<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/Dynamic_Circuits_Coding_with_Qiskit_1_x_Programming_on_Quantum_Computers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 -m pip install qiskit[visualization]
!python3 -m pip install qiskit-ibm-runtime
!python3 -m pip install qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 4.4 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=f04ea002c867ab095b9672dd641bb85978508939e48079d7fcb00e316a01b2b5
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

## Long-range CNOT gate teleportation using dynamic circuits
# Setp 1: Map the problem to curcuits and operators

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr

In [10]:
def get_dynamic_CNOT_curcuit(num_qubit):
  """
  (1) 1D chain of nearest neighbors
  (2) 0th qubit is the control, and the last qubit (num_qubit) is the target
  (3) The control qubit starts in the + states
  """
  num_ancilla = num_qubit + 2
  num_ancilla_pair = int(num_ancilla / 2)

  qr = QuantumRegister(num_qubit)
  cr1 = ClassicalRegister(num_ancilla_pair, name = "cr1") # The parity-controlled X gate
  cr2 = ClassicalRegister(num_ancilla - num_ancilla_pair, name="cr2") # The parity-controlled Z gate
  cr3 = ClassicalRegister(2, name="cr3") # For the final measurements on the control and target qubits
  qc = QuantumCircuit(qr, cr1, cr2, cr3)

  #Initialize the control qubits
  qc.h(0)
  qc.barrier()

  # Entangle the control qubit and the first ancilla qubit
  qc.cx(0, 1)

  # Create Bell pairs on ancilla qubits
  # The first ancilla qubit in index 1
  for i in range(num_ancilla_pair):
    qc.h(2+2*i)
    qc.cx(2+2*i, 2+2*i+1)
  ###for

  # Prepare Bell pairs on staggered ancilla and data qubits
  for i in range(num_ancilla_pair+1):
    qc.cx(1+2*i, 1+2*i+1)
  ###for

  for i in range(1, num_ancilla_pair+2):
    qc.h(2*i-1)
  ###for

  # Measurement on alternating ancilla qubits starting with the first one
  # Keep track of the parity for eventual conditional Z gate
  for i in range(1, num_ancilla_pair+2):
    qc.measure(2*i-1, cr2[i-1])
    if i == 1:
      parity_control = expr.lift(cr2[i-1])
    else:
      parity_control = expr.bit_xor(cr2[i-1], parity_control)
    ###if
  ###for

  # Measurement on staggered alternating ancilla qubits starting with the second
  # Keep track of the parity of eventual conditional X gate
  for i in range(num_ancilla_pair):
    qc.measure(2*i + 2, cr1[i])
    if i == 0:
      parity_target = expr.lift(cr1[i])
    else:
      parity_target = expr.bit_xor(cr1[i], parity_target)
    ###if
  ###for
  with qc.if_test(parity_control):
    qc.z(0)
  ###with

  with qc.if_test(parity_target):
    qc.x(-1)
  ###with
  return qc
###def

In [11]:
qc = get_dynamic_CNOT_curcuit(num_qubit=7)
qc.draw(output='mpl')

CircuitError: 'Index 7 out of range for size 7.'

In [13]:
max_num_qubit = 41

qc_list = []
num_qubit_list = list(range(7, max_num_qubit+1, 2))
for num_qubit in num_qubit_list:
  qc_list.append(get_dynamic_CNOT_curcuit(num_qubit))
###for

CircuitError: 'Index 7 out of range for size 7.'

## Step 2: Optimize the problem for quantum execution

In [14]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService

backend_name = "ibm_cusco"
service = QiskitRuntimeService()
backend = service.get_backend(backend_name)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

qc_transpilid_list = pm.run(qc_list)


AccountNotFoundError: 'Unable to find account.'

## Step 3: Execute the circuit

In [15]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

sampler = Sampler(backend=backend)
job = sampler.run(qc_transpilid_list)
print(job.job_id)

NameError: name 'backend' is not defined

## Step 4: Post-processing ( and plotting)

In [16]:
import matplotlib.pyplot as plt
from qiskit_ibm_runtime import QiskitRuntimeService

job_id = ' ' # Cusco, documentation providier, max_qubit=41, new image

service = QiskitRuntimeService()
job = service.jbo(job_id)
result = job.result()

list_Bell = []
list_other = []
for i in range(0, len(qc_list)):
  data = result[i+1].data
  counts = data.cr3.get_counts()
  total_counts = data.cr3.num_shots

  prob_Bell = (counts['00'] + counts['11']) / total_counts

  list_Bell.append(prob_Bell)
  list_other.append(1-prob_Bell)
###for

plt.plot(num_qubit_lsit, list_Bell, '--o', label='00 or 11')
plt.plot(num_qubit_list, list_other, '-.^', label='other')
plt.xlabel('Number of qubits')
plt.ylabel('Probability')
plt.legend()


AccountNotFoundError: 'Unable to find account.'